Filling in blanks

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [0]:
def count(dataframe):
  print(np.logical_not(dataframe.isnull()).sum()[0])

In [0]:
def load_data(path):
  df = pd.read_csv(path,sep = ';')
  return df

In [0]:
df = load_data('/content/drive/My Drive/BazeField/bessakerdata16101246_30sec.csv')

In [276]:
df = df.copy()
df.head()

,TimeStamp,BESS-WTG01-WindVane (Average),BESS-WTG01-WindSpeed (Average),BESS-WTG01-RotorSpeed (Average),BESS-WTG01-NacelleDirection (Average),BESS-WTG01-ActivePower (Average),BESS-WTG01-Spinner-Temperature (Average),BESS-WTG01-Log-T-Raw-FrontBearingTemperature (Average),BESS-WTG01-Log-T-Raw-RearBearingTemperature (Average),BESS-WTG01-Log-T-Raw-BladeAPitchHeatSink (Average),BESS-WTG01-Log-T-Raw-BladeBPitchHeatSink (Average),BESS-WTG01-Log-T-Raw-BladeCPitchHeatSink (Average),BESS-WTG01-Log-T-Raw-BladeAPitchControlBox (Average),BESS-WTG01-Log-T-Raw-BladeBPitchControlBox (Average),BESS-WTG01-Log-T-Raw-BladeCPitchControlBox (Average),BESS-WTG01-Log-T-Raw-BladeATemperature (Average),BESS-WTG01-Log-T-Raw-BladeBTemperature (Average),BESS-WTG01-Log-T-Raw-BladeCTemperature (Average),BESS-WTG01-Log-T-Raw-Rotor1Temperature (Average),BESS-WTG01-Log-T-Raw-Rotor2Temperature (Average),BESS-WTG01-Log-T-Raw-Stator2Temperature (Average),BESS-WTG01-Log-T-Raw-Stator1Temperature (Average),BESS-WTG01-Log-T-Raw-NacelleAmbientTemperature (Average),BESS-WTG01-Nacelle-Temperature (Average),BESS-WTG01-Log-T-Raw-NacelleControlCabinetTemperature (Average),BESS-WTG01-Log-T-Raw-Rectifier1HeatSink (Average),BESS-WTG01-Log-T-Raw-Rectifier2HeatSink (Average),BESS-WTG01-Log-T-Raw-RectifierCabinetTemperature (Average),BESS-WTG01-Log-T-Raw-ExcitationHeatSink (Average),BESS-WTG01-Ambient-Temperature (Average),BESS-WTG01-Log-T-Raw-TowerTemperature (Average),BESS-WTG01-Log-T-Raw-ControlCabinetTemperature (Average),BESS-WTG01-Log-T-Raw-TransformerTemperature (Average),BESS-WTG01-Average blade angle across A B C (Average),BESS-WTG01-WindDirection (Average)
0,15-07-2017 00:00:00,NaN,4.5,9.75,227.0,109.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN
1,15-07-2017 00:00:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15-07-2017 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15-07-2017 00:01:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15-07-2017 00:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [277]:
# This dataset is a dataset for a SINGLE wind turbine:

# Slice dataset on this row nr.: 177228 (4-09-2017 12:54:00)
#df.iloc[[177228]]


location = np.where(df["TimeStamp"].str.contains("25-09-2017 13:20:00"))
print(location)
split_on_index = location[0][0]
print(split_on_index)

# Splitting
df1 = df.iloc[:split_on_index, :]
df2 = df.iloc[split_on_index:, :]

# Reseting index
df2 = df2.reset_index(drop=True)


df2.head()

(array([208960]),)
208960


,TimeStamp,BESS-WTG01-WindVane (Average),BESS-WTG01-WindSpeed (Average),BESS-WTG01-RotorSpeed (Average),BESS-WTG01-NacelleDirection (Average),BESS-WTG01-ActivePower (Average),BESS-WTG01-Spinner-Temperature (Average),BESS-WTG01-Log-T-Raw-FrontBearingTemperature (Average),BESS-WTG01-Log-T-Raw-RearBearingTemperature (Average),BESS-WTG01-Log-T-Raw-BladeAPitchHeatSink (Average),BESS-WTG01-Log-T-Raw-BladeBPitchHeatSink (Average),BESS-WTG01-Log-T-Raw-BladeCPitchHeatSink (Average),BESS-WTG01-Log-T-Raw-BladeAPitchControlBox (Average),BESS-WTG01-Log-T-Raw-BladeBPitchControlBox (Average),BESS-WTG01-Log-T-Raw-BladeCPitchControlBox (Average),BESS-WTG01-Log-T-Raw-BladeATemperature (Average),BESS-WTG01-Log-T-Raw-BladeBTemperature (Average),BESS-WTG01-Log-T-Raw-BladeCTemperature (Average),BESS-WTG01-Log-T-Raw-Rotor1Temperature (Average),BESS-WTG01-Log-T-Raw-Rotor2Temperature (Average),BESS-WTG01-Log-T-Raw-Stator2Temperature (Average),BESS-WTG01-Log-T-Raw-Stator1Temperature (Average),BESS-WTG01-Log-T-Raw-NacelleAmbientTemperature (Average),BESS-WTG01-Nacelle-Temperature (Average),BESS-WTG01-Log-T-Raw-NacelleControlCabinetTemperature (Average),BESS-WTG01-Log-T-Raw-Rectifier1HeatSink (Average),BESS-WTG01-Log-T-Raw-Rectifier2HeatSink (Average),BESS-WTG01-Log-T-Raw-RectifierCabinetTemperature (Average),BESS-WTG01-Log-T-Raw-ExcitationHeatSink (Average),BESS-WTG01-Ambient-Temperature (Average),BESS-WTG01-Log-T-Raw-TowerTemperature (Average),BESS-WTG01-Log-T-Raw-ControlCabinetTemperature (Average),BESS-WTG01-Log-T-Raw-TransformerTemperature (Average),BESS-WTG01-Average blade angle across A B C (Average),BESS-WTG01-WindDirection (Average)
0,25-09-2017 13:20:00,18.5,5.1,13.27,123.0,311.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.500000
1,25-09-2017 13:20:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25-09-2017 13:21:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25-09-2017 13:21:30,16.9,NaN,12.95,124.0,276.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.899994
4,25-09-2017 13:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Find index of first value in a column that is not NaN:

In [0]:
# Find the first non-NaN value in a column
def find_first_not_nan(df,column_name):
  ser = df2[column_name].notnull()
  length = ser.shape[0]

  for i in range (length):
    if (ser[i] == True):
      return i
  return "no values in column!"

## Loop through all columns in the dataset and put in the first not NaN number



In [0]:
# Looping all columns in dataframe
def loop_through_and_fill_first_val(data):
  for col in data.columns:

    index = find_first_not_nan(data,col)

    if (index != 0):
      # Put the value at the found index at the first index:
      data.at[0, col] = data.at[index, col]

    # Dropping columns if the first value is too far in the data set
    if (index > 200):
      data.drop([col], axis=1,inplace = True)

In [0]:
loop_through_and_fill_first_val(df2)

In [281]:
df2.head()


,TimeStamp,BESS-WTG01-WindVane (Average),BESS-WTG01-WindSpeed (Average),BESS-WTG01-RotorSpeed (Average),BESS-WTG01-NacelleDirection (Average),BESS-WTG01-ActivePower (Average),BESS-WTG01-Spinner-Temperature (Average),BESS-WTG01-Log-T-Raw-FrontBearingTemperature (Average),BESS-WTG01-Log-T-Raw-RearBearingTemperature (Average),BESS-WTG01-Log-T-Raw-BladeAPitchHeatSink (Average),BESS-WTG01-Log-T-Raw-BladeBPitchHeatSink (Average),BESS-WTG01-Log-T-Raw-BladeCPitchHeatSink (Average),BESS-WTG01-Log-T-Raw-BladeAPitchControlBox (Average),BESS-WTG01-Log-T-Raw-BladeBPitchControlBox (Average),BESS-WTG01-Log-T-Raw-BladeCPitchControlBox (Average),BESS-WTG01-Log-T-Raw-BladeATemperature (Average),BESS-WTG01-Log-T-Raw-BladeBTemperature (Average),BESS-WTG01-Log-T-Raw-Rotor1Temperature (Average),BESS-WTG01-Log-T-Raw-Rotor2Temperature (Average),BESS-WTG01-Log-T-Raw-Stator2Temperature (Average),BESS-WTG01-Log-T-Raw-Stator1Temperature (Average),BESS-WTG01-Log-T-Raw-NacelleAmbientTemperature (Average),BESS-WTG01-Nacelle-Temperature (Average),BESS-WTG01-Log-T-Raw-NacelleControlCabinetTemperature (Average),BESS-WTG01-Log-T-Raw-Rectifier1HeatSink (Average),BESS-WTG01-Log-T-Raw-Rectifier2HeatSink (Average),BESS-WTG01-Log-T-Raw-RectifierCabinetTemperature (Average),BESS-WTG01-Log-T-Raw-ExcitationHeatSink (Average),BESS-WTG01-Ambient-Temperature (Average),BESS-WTG01-Log-T-Raw-TowerTemperature (Average),BESS-WTG01-Log-T-Raw-ControlCabinetTemperature (Average),BESS-WTG01-Log-T-Raw-TransformerTemperature (Average),BESS-WTG01-WindDirection (Average)
0,25-09-2017 13:20:00,18.5,5.1,13.27,123.0,311.0,31.0,41.0,37.0,39.0,40.0,40.0,55.0,54.0,54.0,165.0,164.0,60.0,60.0,62.0,63.0,21.0,36.0,36.0,46.0,52.0,49.0,46.0,23.0,34.0,41.0,52.0,141.500000
1,25-09-2017 13:20:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25-09-2017 13:21:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25-09-2017 13:21:30,16.9,NaN,12.95,124.0,276.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.899994
4,25-09-2017 13:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Filling all missing values

In [0]:
def fill_missing_values(dataframe):
  # Filling in the missing values:
  dataframe.fillna(method='ffill', inplace=True)
  dataframe.head()
  return dataframe

In [0]:
df2 = fill_missing_values(df2)

# The Error log

### Reading the data for WT01

In [0]:
df_log = pd.read_excel('/content/drive/My Drive/BazeField/alarmlogs/AlarmlogBessaker.xlsx')

In [305]:
# Check that the error set only contains "Alarms" and no "Warnings"
df_log.Type.unique()

array(['Alarm'], dtype=object)

### Filtering and sorting errors by date

In [306]:
df_log_WT01 = df_log[df_log.Turbine == "BESS-WTG01"]
df_log_WT01
df_log_WT01.sort_values(by=['Start'], inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Removing errors older than 25-09-2017

In [307]:
from datetime import date
df_log_WT01["Start"] = pd.to_datetime(df_log_WT01['Start'])
df_log_WT01.head()
df_log_WT01 = df_log_WT01[(df_log_WT01.Start >= date(2017, 9, 25))]
df_log_WT01

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.


,Site,Turbine,Alarm,Code,Event Type,Description,Start,End,Duration,Type,Unnamed: 10
20061,Bessaker,BESS-WTG01,Status_8_Substatus_0,0.333333,A,N 8:0 Maintenance - (Maintenance),2017-09-26 12:00:46,2017-09-26 13:21:36,01:20:50,Alarm,NaN
19884,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),2017-10-03 14:17:20,2017-10-03 15:09:49,00:52:29,Alarm,NaN
19863,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),2017-10-03 15:09:49,2017-10-03 15:52:04,00:42:15,Alarm,NaN
19857,Bessaker,BESS-WTG01,Status_21_Substatus_2,0.876389,A,N 21:2 Cable twisted - (Right (2-3 turns)),2017-10-03 15:52:04,2017-10-03 16:29:50,00:37:46,Alarm,NaN
19851,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),2017-10-03 16:29:50,2017-10-03 19:21:11,02:51:21,Alarm,NaN
...,...,...,...,...,...,...,...,...,...,...,...
146,Bessaker,BESS-WTG01,Status_17_Substatus_0,0.708333,A,N 17:0 Test safety system - (Test safety sys...,2019-10-02 10:00:04,2019-10-02 10:01:04,00:01:00,Alarm,NaN
145,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),2019-10-02 10:01:04,2019-10-02 10:06:16,00:05:12,Alarm,NaN
138,Bessaker,BESS-WTG01,Status_240_Substatus_1,240:1,A,R 240:1 STATUS: Remote control PC (switched off),2019-10-02 12:24:29,2019-10-02 12:26:37,00:02:08,Alarm,NaN
98,Bessaker,BESS-WTG01,Status_240_Substatus_0,240:0,A,R 240:0 STATUS: Remote control PC (switched on),2019-10-02 12:26:37,2019-10-02 12:26:43,00:00:06,Alarm,NaN


### Formatting the dates in the error data

In [0]:

# What I have: format='%Y-%m-%d %H:%M:%S').
# What I want: '%d-%m-%Y %H:%M:%S'

def format_dates(dataframe):
  dataframe['Start'] = pd.to_datetime(dataframe['Start'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%d-%m-%Y %H:%M:%S')
  dataframe['End'] = pd.to_datetime(dataframe['End'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%d-%m-%Y %H:%M:%S')
  return dataframe

In [309]:
df_log_WT01 = format_dates(df_log_WT01)

df_log_WT01.head()

,Site,Turbine,Alarm,Code,Event Type,Description,Start,End,Duration,Type,Unnamed: 10
20061,Bessaker,BESS-WTG01,Status_8_Substatus_0,0.333333,A,N 8:0 Maintenance - (Maintenance),26-09-2017 12:00:46,26-09-2017 13:21:36,01:20:50,Alarm,NaN
19884,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),03-10-2017 14:17:20,03-10-2017 15:09:49,00:52:29,Alarm,NaN
19863,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),03-10-2017 15:09:49,03-10-2017 15:52:04,00:42:15,Alarm,NaN
19857,Bessaker,BESS-WTG01,Status_21_Substatus_2,0.876389,A,N 21:2 Cable twisted - (Right (2-3 turns)),03-10-2017 15:52:04,03-10-2017 16:29:50,00:37:46,Alarm,NaN
19851,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),03-10-2017 16:29:50,03-10-2017 19:21:11,02:51:21,Alarm,NaN


### Filtering the wanted errors

In [0]:
'''
Error descriptions:

    "Status_9_Substatus_8" = Generator heating - (Manual)
    "Status_62_Substatus_7" = Feeding fault - (Diff. P-set/P-actual)
    "Status_15_Substatus_1" = Turbine moist - (Turbine moist Inverter 1)
    "Status_62_Substatus_30" = Feeding fault - (Feeding safety circuit faulty)
 
    '''
selected_errors = [
    "Status_9_Substatus_8",
    "Status_62_Substatus_7",
    "Status_62_Substatus_30"

]

def remove_unnecessary_faults(dataframe):
  new_df = pd.DataFrame(columns=df_log_WT01.columns)

  for fault in selected_errors:
    new_df = new_df.append(df_log_WT01[ df_log_WT01["Alarm"] == fault])

  return new_df

In [328]:
# df_log_WT01 = remove_unnecessary_faults(df_log_WT01)

df_log_WT01.sort_values(by=['Start'], inplace = True)
df_log_WT01.reset_index(drop=True)

,Site,Turbine,Alarm,Code,Event Type,Description,Start,End,Duration,Type,Unnamed: 10
0,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),01-06-2018 00:29:57,01-06-2018 00:47:04,00:17:07,Alarm,NaN
1,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),01-06-2018 00:47:04,01-06-2018 01:12:56,00:25:52,Alarm,NaN
2,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),01-06-2018 03:05:32,01-06-2018 03:44:11,00:38:39,Alarm,NaN
3,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),01-06-2018 04:12:08,01-06-2018 04:22:23,00:10:15,Alarm,NaN
4,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),01-06-2018 04:59:36,01-06-2018 05:41:04,00:41:28,Alarm,NaN
...,...,...,...,...,...,...,...,...,...,...,...
666,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),31-05-2018 22:16:52,31-05-2018 22:23:48,00:06:56,Alarm,NaN
667,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),31-08-2018 05:09:07,31-08-2018 06:19:04,01:09:57,Alarm,NaN
668,Bessaker,BESS-WTG01,Status_21_Substatus_2,0.876389,A,N 21:2 Cable twisted - (Right (2-3 turns)),31-08-2018 14:55:57,31-08-2018 15:34:03,00:38:06,Alarm,NaN
669,Bessaker,BESS-WTG01,Status_2_Substatus_1,0.0840278,A,N 2:1 Lack of wind - (Wind speed too low),31-08-2018 15:34:03,31-08-2018 15:46:59,00:12:56,Alarm,NaN


In [329]:
start_time = df_log_WT01['Start'].iloc[200]
end_time = df_log_WT01['End'].iloc[200]

print(start_time)
print(end_time)

10-09-2018 16:41:23
10-09-2018 17:27:17


### Inserting new column with 0 as value

In [331]:
df['Status'] = 0

,TimeStamp,BESS-WTG01-WindVane (Average),BESS-WTG01-WindSpeed (Average),BESS-WTG01-RotorSpeed (Average),BESS-WTG01-NacelleDirection (Average),BESS-WTG01-ActivePower (Average),BESS-WTG01-Spinner-Temperature (Average),BESS-WTG01-Log-T-Raw-FrontBearingTemperature (Average),BESS-WTG01-Log-T-Raw-RearBearingTemperature (Average),BESS-WTG01-Log-T-Raw-BladeAPitchHeatSink (Average),BESS-WTG01-Log-T-Raw-BladeBPitchHeatSink (Average),BESS-WTG01-Log-T-Raw-BladeCPitchHeatSink (Average),BESS-WTG01-Log-T-Raw-BladeAPitchControlBox (Average),BESS-WTG01-Log-T-Raw-BladeBPitchControlBox (Average),BESS-WTG01-Log-T-Raw-BladeCPitchControlBox (Average),BESS-WTG01-Log-T-Raw-BladeATemperature (Average),BESS-WTG01-Log-T-Raw-BladeBTemperature (Average),BESS-WTG01-Log-T-Raw-BladeCTemperature (Average),BESS-WTG01-Log-T-Raw-Rotor1Temperature (Average),BESS-WTG01-Log-T-Raw-Rotor2Temperature (Average),BESS-WTG01-Log-T-Raw-Stator2Temperature (Average),BESS-WTG01-Log-T-Raw-Stator1Temperature (Average),BESS-WTG01-Log-T-Raw-NacelleAmbientTemperature (Average),BESS-WTG01-Nacelle-Temperature (Average),BESS-WTG01-Log-T-Raw-NacelleControlCabinetTemperature (Average),BESS-WTG01-Log-T-Raw-Rectifier1HeatSink (Average),BESS-WTG01-Log-T-Raw-Rectifier2HeatSink (Average),BESS-WTG01-Log-T-Raw-RectifierCabinetTemperature (Average),BESS-WTG01-Log-T-Raw-ExcitationHeatSink (Average),BESS-WTG01-Ambient-Temperature (Average),BESS-WTG01-Log-T-Raw-TowerTemperature (Average),BESS-WTG01-Log-T-Raw-ControlCabinetTemperature (Average),BESS-WTG01-Log-T-Raw-TransformerTemperature (Average),BESS-WTG01-Average blade angle across A B C (Average),BESS-WTG01-WindDirection (Average),Status
0,15-07-2017 00:00:00,NaN,4.5,9.75,227.0,109.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,0
1,15-07-2017 00:00:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,15-07-2017 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,15-07-2017 00:01:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,15-07-2017 00:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [0]:
# Start: 16:41:23. Need to change the status from and including 16:41:00 
# End: 17:27:17. Need to change the status to and including 17:27:30

### Splitting the error data on the wanted date (25-09-2017)

In [0]:
import seaborn as sns

fig = plt.figure(figsize = (12,6))
sns.countplot(x='TimeStamp',data = removed_columns)
plt.show()